# 1. IMPORTS

In [1]:
import os,pandas,sys,time,keras,json,sklearn,tensorflow,random
import numpy as np
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ['PYTHONHASHSEED']='0'

def reseed():
    seed = 0x5f3759df
    np.random.seed(seed)
    tensorflow.random.set_seed(seed)
    keras.utils.set_random_seed(seed)
    random.seed(seed)

from starter import Starter
starter = Starter()
starter.start(lambda: os.chdir(os.path.dirname(os.getcwd())))

2025-05-30 08:48:37.746385: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-30 08:48:37.746852: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-30 08:48:37.749525: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-30 08:48:37.757345: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748569717.769981  545260 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748569717.77

Starter has been initialized.


# 2. LOAD DATA

In [2]:
#!/usr/bin/env python3
from utils.data_loader import DataLoader
import numpy as np 

print("1. LOADING AND PREPROCESSING DATA")
print("="*50)

# Initialize data loader
data_dir = "../data" 
data_loader = DataLoader(data_dir)

# Load and prepare data
try:
    X_train, y_train, X_valid, y_valid, X_test, y_test = data_loader.prepare_data(
        max_vocab_size=5000,
        max_length=50,
    )
    
    print("\nData loaded successfully!")
    print(f"Vocabulary size: {data_loader.preprocessor.vocab_size}")
    print(f"Number of classes: {data_loader.num_classes}")
    print(f"Max sequence length: {data_loader.preprocessor.max_length}")
    
    # Show class distribution
    unique, counts = np.unique(y_train, return_counts=True)
    print("\nClass distribution in training data:")
    for class_id, count in zip(unique, counts):
        class_name = data_loader.reverse_label_encoder[class_id]
        print(f"  {class_name}: {count} ({count/len(y_train)*100:.1f}%)")
        
    print("\nData shapes:")
    print(f"  Training: X={X_train.shape}, y={y_train.shape}")
    print(f"  Validation: X={X_valid.shape}, y={y_valid.shape}")
    print(f"  Test: X={X_test.shape}, y={y_test.shape}")
    
    # Display sample data
    print("\nSample data:")
    print(f"  First training text tokens: {X_train[0][:10]}...")
    print(f"  First training label: {y_train[0]} ({data_loader.reverse_label_encoder[y_train[0]]})")
    
    # Verify data integrity
    print("\nData integrity checks:")
    print(f"  No missing values in X_train: {not np.any(np.isnan(X_train))}")
    print(f"  No missing values in y_train: {not np.any(np.isnan(y_train))}")
    print(f"  All labels in valid range: {np.all((y_train >= 0) & (y_train < data_loader.num_classes))}")
    
    print("\nData preprocessing completed successfully!")
        
except Exception as e:
    print(f"Error loading data: {e}")
    import traceback
    traceback.print_exc()
    raise e


1. LOADING AND PREPROCESSING DATA
Loaded data:
  Train: 500 samples
  Valid: 100 samples
  Test: 400 samples
Keras vectorizer vocabulary size: 2836
Sample vocabulary: ['', '[UNK]', np.str_('yang'), np.str_('di'), np.str_('dan'), np.str_('tidak'), np.str_('saya'), np.str_('dengan'), np.str_('enak'), np.str_('ini')]

Data shapes:
  X_train: (500, 50)
  y_train: (500,)
  X_valid: (100, 50)
  y_valid: (100,)
  X_test: (400, 50)
  y_test: (400,)

Data loaded successfully!
Vocabulary size: 2836
Number of classes: 3
Max sequence length: 50

Class distribution in training data:
  negative: 192 (38.4%)
  neutral: 119 (23.8%)
  positive: 189 (37.8%)

Data shapes:
  Training: X=(500, 50), y=(500,)
  Validation: X=(100, 50), y=(100,)
  Test: X=(400, 50), y=(400,)

Sample data:
  First training text tokens: [1758 1080 1145  196 2834  198   11  607  177  847]...
  First training label: 1 (neutral)

Data integrity checks:
  No missing values in X_train: True
  No missing values in y_train: True
  All

2025-05-30 08:48:39.528996: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


# 3. INITIALIZE EXPERIMENT RUNNER

In [3]:
#!/usr/bin/env python3
import os
import time
import json
import numpy as np
import keras
import sklearn.metrics


class KerasLSTMExperiment:
    """Keras LSTM experiment class for systematic hyperparameter analysis"""
    
    def __init__(self, data_loader, X_train, y_train, X_valid, y_valid, X_test, y_test):
        self.data_loader = data_loader
        self.X_train = X_train
        self.y_train = y_train
        self.X_valid = X_valid
        self.y_valid = y_valid
        self.X_test = X_test
        self.y_test = y_test
        
        # Base configuration
        self.base_config = {
            'vocab_size': data_loader.preprocessor.vocab_size,
            'embedding_dim': 64,
            'lstm_units': 32,
            'num_classes': data_loader.num_classes,
            'max_length': data_loader.preprocessor.max_length,
            'activation': 'tanh',
            'dropout_rate': 0.2,
            'learning_rate': 0.001,
            'batch_size': 32,
            'epochs': 15
        }
        
        print(f"Base configuration:")
        for key, value in self.base_config.items():
            print(f"  {key}: {value}")
    
    def create_keras_model(self, config):
        """Create Keras lstm model with given configuration"""
        reseed()
        model = keras.models.Sequential()
        model.add(keras.layers.Embedding(
            input_dim=config['vocab_size'],
            output_dim=config['embedding_dim'],
            input_length=config['max_length'],
            name='embedding'
        ))
        
        for i in range(config['num_lstm_layers']):
            return_sequences = i < config['num_lstm_layers'] - 1
            lstm_layer = keras.layers.LSTM(
                units=config['lstm_units'],
                activation=config['activation'],
                return_sequences=return_sequences,
                name=f'lstm_{i}'
            )
            
            if config['bidirectional']:
                model.add(keras.layers.Bidirectional(lstm_layer, name=f'bidirectional_lstm_{i}'))
            else:
                model.add(lstm_layer)
            
            if i < config['num_lstm_layers'] - 1:
                model.add(keras.layers.Dropout(config['dropout_rate'], name=f'dropout_{i}'))
        
        model.add(keras.layers.Dropout(config['dropout_rate'], name='dropout_final'))
        model.add(keras.layers.Dense(config['num_classes'], activation='softmax', name='classification'))
        
        model.build(input_shape=(None, config['max_length']))
        
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=config['learning_rate']),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        
        return model
    
    def train_and_evaluate(self, config, experiment_name):
        print(f"\n{'='*60}")
        print(f"Training: {experiment_name}")
        print(f"Config: {config}")
        print(f"{'='*60}")
        
        start_time = time.time()
        model = self.create_keras_model(config)
        print(f"Model created with {model.count_params():,} parameters")
        
        print(f"Starting training for {config['epochs']} epochs...")
        history = model.fit(
            self.X_train, self.y_train,
            validation_data=(self.X_valid, self.y_valid),
            epochs=config['epochs'],
            batch_size=config['batch_size'],
            verbose=1,
            callbacks=[
                keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=5,
                    restore_best_weights=True
                )
            ],
            shuffle=False
        )
        
        print(f"Evaluating on test set...")
        test_loss, test_acc = model.evaluate(self.X_test, self.y_test, verbose=0)
        test_predictions = model.predict(self.X_test, verbose=0)
        test_pred_classes = np.argmax(test_predictions, axis=1)
        test_f1_macro = sklearn.metrics.f1_score(self.y_test, test_pred_classes, average='macro')
        
        valid_predictions = model.predict(self.X_valid, verbose=0)
        valid_pred_classes = np.argmax(valid_predictions, axis=1)
        valid_f1_macro = sklearn.metrics.f1_score(self.y_valid, valid_pred_classes, average='macro')
        
        weights_path = f"results/{experiment_name}_weights.npz"
        os.makedirs("results", exist_ok=True)
        self.save_keras_weights(model, weights_path, config)

        unique_rows = np.unique(self.X_test, axis=0)

        
        training_time = time.time() - start_time
        
        print(f"\nResults for {experiment_name}:")
        print(f"  Training time: {training_time:.2f} seconds")
        print(f"  Test Accuracy: {test_acc:.4f}")
        print(f"  Test F1-Score (macro): {test_f1_macro:.4f}")
        print(f"  Valid F1-Score (macro): {valid_f1_macro:.4f}")
        print(f"  Weights saved to: {weights_path}")
        
        return {
            'model': model,
            'history': history.history,
            'test_accuracy': test_acc,
            'test_f1_score': test_f1_macro,
            'valid_f1_score': valid_f1_macro,
            'weights_path': weights_path,
            'config': config,
            'training_time': training_time
        }
    
    def save_keras_weights(self, model, filepath, config):
        print(f"Saving Keras weights to: {filepath}")
        
        try:
            if len(model.weights) == 0:
                raise ValueError("Model has no weights to save!")
            
            weights_dict = {}
            lstm_layer_count = 0
            
            for layer in model.layers:
                layer_weights = layer.get_weights()
                if len(layer_weights) == 0:
                    continue
                    
                layer_name = layer.name
                print(f"  Processing layer: {layer_name} - {len(layer_weights)} weight arrays")
                
                if 'embedding' in layer_name.lower():
                    weights_dict['embedding'] = {
                        'embedding_matrix': layer_weights[0]
                    }
                elif 'simple_lstm' in layer_name.lower():
                    target_name = f'lstm_{lstm_layer_count}'
                    weights_dict[target_name] = {
                        'W_ih': layer_weights[0].T,
                        'W_hh': layer_weights[1].T,
                        'b_h': layer_weights[2]
                    }
                    lstm_layer_count += 1
                elif 'bidirectional' in layer_name.lower():
                    target_name = f'bidirectional_lstm_{lstm_layer_count}'
                    if len(layer_weights) >= 6:
                        weights_dict[target_name] = {
                            'forward_W_ih': layer_weights[0].T,
                            'forward_W_hh': layer_weights[1].T,
                            'forward_b_h': layer_weights[2],
                            'backward_W_ih': layer_weights[3].T,
                            'backward_W_hh': layer_weights[4].T,
                            'backward_b_h': layer_weights[5]
                        }
                    lstm_layer_count += 1
                elif 'dense' in layer_name.lower() or 'classification' in layer_name.lower():
                    weights_dict['classification'] = {
                        'W': layer_weights[0].T,
                        'b': layer_weights[1]
                    }
            
            save_dict = {}
            for layer_name, layer_weights in weights_dict.items():
                for weight_name, weight_value in layer_weights.items():
                    save_dict[f"{layer_name}_{weight_name}"] = weight_value
            
            save_dict['config'] = json.dumps(config)
            np.savez(filepath, **save_dict)
            print(f"  Saved {len(save_dict)-1} weight arrays successfully")
            print(f"  Layers saved: {list(weights_dict.keys())}")
            
            loaded_check = np.load(filepath)
            assert len(loaded_check.files) == len(save_dict), "Save verification failed"
            loaded_check.close()
        
        except Exception as e:
            print(f"Error saving weights: {e}")
            import traceback
            traceback.print_exc()
            raise


print("Initializing Keras experiment framework...")
keras_experiment = KerasLSTMExperiment(
    data_loader, X_train, y_train, X_valid, y_valid, X_test, y_test
)
print("Keras experiment runner initialized!")


Initializing Keras experiment framework...
Base configuration:
  vocab_size: 2836
  embedding_dim: 64
  lstm_units: 32
  num_classes: 3
  max_length: 50
  activation: tanh
  dropout_rate: 0.2
  learning_rate: 0.001
  batch_size: 32
  epochs: 15
Keras experiment runner initialized!


# 4. EXPERIMENTS

## 4.1. Comparing Layer Counts

In [4]:
#!/usr/bin/env python3
layer_counts = [1, 3, 5]
layer_results = {}

for num_layers in layer_counts:
    config = keras_experiment.base_config.copy()
    config.update({
        'num_lstm_layers': num_layers,
        'bidirectional': False, 
        'lstm_units': 64
    })
    # config['epochs'] = 10
    
    experiment_name = f"lstm_layers_{num_layers}"
    print(f"\n🔬 Experiment: {experiment_name}")
    print(f"   Layers: {num_layers}, Units: {config['lstm_units']}, Bidirectional: {config['bidirectional']}")
    
    try:
        result = keras_experiment.train_and_evaluate(config, experiment_name)
        for k,v in result.items():
            print(k, v)
        layer_results[num_layers] = result
        print(f" {experiment_name} completed successfully!")
    except Exception as e:
        print(f" Error in {experiment_name}: {e}")
        continue

print(f"{'Layers':<8} {'Accuracy':<10} {'F1-Score':<10} {'Time (s)':<10}")
print("-"*40)
for layers in sorted(layer_results.keys()):
    result = layer_results[layers]
    print(f"{layers:<8} {result['test_accuracy']:.4f}    {result['test_f1_score']:.4f}    {result['training_time']:.1f}")



🔬 Experiment: lstm_layers_1
   Layers: 1, Units: 64, Bidirectional: False

Training: lstm_layers_1
Config: {'vocab_size': 2836, 'embedding_dim': 64, 'lstm_units': 64, 'num_classes': 3, 'max_length': 50, 'activation': 'tanh', 'dropout_rate': 0.2, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 15, 'num_lstm_layers': 1, 'bidirectional': False}


/home/ganadipa/code/kuliah/sem6/Machine-learning/Tugas-Besar-II-ML/.venv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model created with 214,723 parameters
Starting training for 15 epochs...
Epoch 1/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.3686 - loss: 1.0971 - val_accuracy: 0.3700 - val_loss: 1.0841
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3950 - loss: 1.0878 - val_accuracy: 0.3800 - val_loss: 1.0792
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4254 - loss: 1.0737 - val_accuracy: 0.5100 - val_loss: 1.0066
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4726 - loss: 0.9811 - val_accuracy: 0.5200 - val_loss: 0.9588
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5746 - loss: 0.8601 - val_accuracy: 0.6000 - val_loss: 0.8726
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6684 - loss: 0.6973 - val_accuracy: 0.6000 - val_loss: 0.9774
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7064 - loss: 0.5931 - val_accuracy: 0.6000 - val_loss: 0.8925
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 

/home/ganadipa/code/kuliah/sem6/Machine-learning/Tugas-Besar-II-ML/.venv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - accuracy: 0.4059 - loss: 1.0929 - val_accuracy: 0.3800 - val_loss: 1.0770
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.3699 - loss: 1.0499 - val_accuracy: 0.5000 - val_loss: 0.9580
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.5716 - loss: 0.8882 - val_accuracy: 0.5600 - val_loss: 0.9136
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.6477 - loss: 0.7373 - val_accuracy: 0.6500 - val_loss: 0.8700
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.7814 - loss: 0.5313 - val_accuracy: 0.6600 - val_loss: 0.9148
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.8550 - loss: 0.4072 - val_accuracy: 0.6400 - val_loss: 1.0606
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.8871 - loss: 0.3785 - val_accuracy: 0.6100 - val_loss: 1.0191
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.9223 - loss: 0.2769 - val_accuracy: 0.5900 - val_loss: 1.

/home/ganadipa/code/kuliah/sem6/Machine-learning/Tugas-Besar-II-ML/.venv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 80ms/step - accuracy: 0.4156 - loss: 1.0746 - val_accuracy: 0.4900 - val_loss: 1.0502
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.5665 - loss: 0.9044 - val_accuracy: 0.5700 - val_loss: 0.9086
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.6886 - loss: 0.7088 - val_accuracy: 0.5500 - val_loss: 0.8645
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 0.8012 - loss: 0.4902 - val_accuracy: 0.6500 - val_loss: 0.9746
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.9281 - loss: 0.2654 - val_accuracy: 0.5700 - val_loss: 1.2943
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.9350 - loss: 0.1851 - val_accuracy: 0.5800 - val_loss: 1.2969
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.9199 - loss: 0.2671 - val_accuracy: 0.5600 - val_loss: 1.1834
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.9126 - loss: 0.2424 - val_accuracy: 0.6300 - val_loss: 1.

## 4.2. Comparing Cell/Unit Counts

In [5]:
#!/usr/bin/env python3
cell_counts = [16, 32, 64, 128, 256]
cell_results = {}

for num_cells in cell_counts:
    config = keras_experiment.base_config.copy()
    config.update({
        'num_lstm_layers': 1,
        'bidirectional': False, 
        'lstm_units': num_cells
    })
    experiment_name = f"lstm_cells_{num_cells}"
    
    try:
        result = keras_experiment.train_and_evaluate(config, experiment_name)
        cell_results[num_cells] = result
        print(f" {experiment_name} completed successfully!")
    except Exception as e:
        print(f" Error in {experiment_name}: {e}")
        continue

print(f"{'Cells':<8} {'Accuracy':<10} {'F1-Score':<10} {'Time (s)':<10}")
print("-"*40)
for cells in sorted(cell_results.keys()):
    result = cell_results[cells]
    print(f"{cells:<8} {result['test_accuracy']:.4f}    {result['test_f1_score']:.4f}    {result['training_time']:.1f}")



Training: lstm_cells_16
Config: {'vocab_size': 2836, 'embedding_dim': 64, 'lstm_units': 16, 'num_classes': 3, 'max_length': 50, 'activation': 'tanh', 'dropout_rate': 0.2, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 15, 'num_lstm_layers': 1, 'bidirectional': False}
Model created with 186,739 parameters
Starting training for 15 epochs...
Epoch 1/15


/home/ganadipa/code/kuliah/sem6/Machine-learning/Tugas-Besar-II-ML/.venv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.3496 - loss: 1.0965 - val_accuracy: 0.4000 - val_loss: 1.0839
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4052 - loss: 1.0881 - val_accuracy: 0.3900 - val_loss: 1.0824
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4466 - loss: 1.0816 - val_accuracy: 0.4100 - val_loss: 1.0806
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4448 - loss: 1.0719 - val_accuracy: 0.3900 - val_loss: 1.0732
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5014 - loss: 1.0383 - val_accuracy: 0.5900 - val_loss: 0.9055
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6760 - loss: 0.7551 - val_accuracy: 0.6100 - val_loss: 0.8684
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7440 - loss: 0.6075 - val_accuracy: 0.6000 - val_loss: 0.9196
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7308 - loss: 0.5557 - val_accuracy: 0.6100 - val_loss: 0.

/home/ganadipa/code/kuliah/sem6/Machine-learning/Tugas-Besar-II-ML/.venv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.3882 - loss: 1.0941 - val_accuracy: 0.3900 - val_loss: 1.0838
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3955 - loss: 1.0875 - val_accuracy: 0.3900 - val_loss: 1.0815
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4054 - loss: 1.0849 - val_accuracy: 0.3700 - val_loss: 1.0772
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4221 - loss: 1.0720 - val_accuracy: 0.4700 - val_loss: 1.0339
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4862 - loss: 0.9631 - val_accuracy: 0.5700 - val_loss: 0.9039
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6119 - loss: 0.7520 - val_accuracy: 0.6100 - val_loss: 0.8598
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7080 - loss: 0.6079 - val_accuracy: 0.6000 - val_loss: 0.9138
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7267 - loss: 0.5548 - val_accuracy: 0.5800 - val_loss: 1.

...

16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.3686 - loss: 1.0971 - val_accuracy: 0.3700 - val_loss: 1.0841
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3950 - loss: 1.0878 - val_accuracy: 0.3800 - val_loss: 1.0792
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4254 - loss: 1.0737 - val_accuracy: 0.5100 - val_loss: 1.0066
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4726 - loss: 0.9811 - val_accuracy: 0.5200 - val_loss: 0.9588
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5746 - loss: 0.8601 - val_accuracy: 0.6000 - val_loss: 0.8726
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6684 - loss: 0.6973 - val_accuracy: 0.6000 - val_loss: 0.9774
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7064 - loss: 0.5931 - val_accuracy: 0.6000 - val_loss: 0.8925
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7325 - loss: 0.5390 - val_accuracy: 0.5300 - val_loss: 1.

/home/ganadipa/code/kuliah/sem6/Machine-learning/Tugas-Besar-II-ML/.venv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.3743 - loss: 1.0936 - val_accuracy: 0.3800 - val_loss: 1.0846
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.3695 - loss: 1.0875 - val_accuracy: 0.3700 - val_loss: 1.0692
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.4505 - loss: 1.0332 - val_accuracy: 0.5100 - val_loss: 0.9996
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5284 - loss: 0.9125 - val_accuracy: 0.5600 - val_loss: 0.8869
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.6854 - loss: 0.6969 - val_accuracy: 0.6100 - val_loss: 0.8480
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7049 - loss: 0.5518 - val_accuracy: 0.6000 - val_loss: 0.8907
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7685 - loss: 0.4925 - val_accuracy: 0.6200 - val_loss: 0.8748
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.8832 - loss: 0.3885 - val_accuracy: 0.4300 - val_loss: 1.

/home/ganadipa/code/kuliah/sem6/Machine-learning/Tugas-Besar-II-ML/.venv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - accuracy: 0.3993 - loss: 1.0945 - val_accuracy: 0.3800 - val_loss: 1.0871
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.3679 - loss: 1.0785 - val_accuracy: 0.4700 - val_loss: 1.0233
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.4989 - loss: 0.9523 - val_accuracy: 0.6000 - val_loss: 0.9648
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.5972 - loss: 0.8302 - val_accuracy: 0.5200 - val_loss: 0.9636
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.6541 - loss: 0.7490 - val_accuracy: 0.6100 - val_loss: 0.9201
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.6913 - loss: 0.6600 - val_accuracy: 0.5500 - val_loss: 1.0314
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.5356 - loss: 0.9193 - val_accuracy: 0.3800 - val_loss: 1.0850
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.3932 - loss: 1.0748 - val_accuracy: 0.3800 - val_loss: 1.

## 4.3. Comparing Bidirectional vs. Unidirectional Layers

In [6]:
#!/usr/bin/env python3
ltype_results = {}
ltype_vars = [False, True]

for ltype in ltype_vars:
    config = keras_experiment.base_config.copy()
    config.update({
        'num_lstm_layers': 1,
        'bidirectional': ltype, 
        'lstm_units': 64
    })
    experiment_name = f"lstm_ltypes_{'bidirectional' if ltype else 'not unidirectional'}"
    
    try:
        result = keras_experiment.train_and_evaluate(config, experiment_name)
        ltype_results[ltype] = result
        print(f" {experiment_name} completed successfully!")
    except Exception as e:
        print(f" Error in {experiment_name}: {e}")
        continue

print(f"{'ltypes':<8} {'Accuracy':<10} {'F1-Score':<10} {'Time (s)':<10}")
print("-"*40)
for ltypes in sorted(ltype_results.keys()):
    result = ltype_results[ltypes]
    print(f"{ltypes:<8} {result['test_accuracy']:.4f}    {result['test_f1_score']:.4f}    {result['training_time']:.1f}")



Training: lstm_ltypes_not unidirectional
Config: {'vocab_size': 2836, 'embedding_dim': 64, 'lstm_units': 64, 'num_classes': 3, 'max_length': 50, 'activation': 'tanh', 'dropout_rate': 0.2, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 15, 'num_lstm_layers': 1, 'bidirectional': False}
Model created with 214,723 parameters
Starting training for 15 epochs...
Epoch 1/15


/home/ganadipa/code/kuliah/sem6/Machine-learning/Tugas-Besar-II-ML/.venv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.3686 - loss: 1.0971 - val_accuracy: 0.3700 - val_loss: 1.0841
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3950 - loss: 1.0878 - val_accuracy: 0.3800 - val_loss: 1.0792
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4254 - loss: 1.0737 - val_accuracy: 0.5100 - val_loss: 1.0066
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4726 - loss: 0.9811 - val_accuracy: 0.5200 - val_loss: 0.9588
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5746 - loss: 0.8601 - val_accuracy: 0.6000 - val_loss: 0.8726
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6684 - loss: 0.6973 - val_accuracy: 0.6000 - val_loss: 0.9774
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7064 - loss: 0.5931 - val_accuracy: 0.6000 - val_loss: 0.8925
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7325 - loss: 0.5390 - val_accuracy: 0.5300 - val_loss: 1.

/home/ganadipa/code/kuliah/sem6/Machine-learning/Tugas-Besar-II-ML/.venv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.3829 - loss: 1.0917 - val_accuracy: 0.4500 - val_loss: 1.0573
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4845 - loss: 1.0033 - val_accuracy: 0.5600 - val_loss: 0.9533
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6284 - loss: 0.8287 - val_accuracy: 0.5700 - val_loss: 0.9066
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7455 - loss: 0.6815 - val_accuracy: 0.6400 - val_loss: 0.8058
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8597 - loss: 0.4432 - val_accuracy: 0.7300 - val_loss: 0.7314
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9538 - loss: 0.2178 - val_accuracy: 0.7700 - val_loss: 0.7170
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9786 - loss: 0.1163 - val_accuracy: 0.7800 - val_loss: 0.7230
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9959 - loss: 0.0626 - val_accuracy: 0.7300 - val_loss: 0.